In [1]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


In [ ]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.read_csv("cau_1.csv")


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Columns: 172 entries, Player to Aerial Duels_Won%
dtypes: float64(135), int64(5), object(32)
memory usage: 662.6+ KB


In [ ]:

columns_to_exclude = ['Player', 'Nation', 'Position', 'Team']

for col in df.columns:
    if col not in columns_to_exclude:
        df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.fillna('N/a')
df.dtypes


Player                            object
Nation                            object
Position                          object
Age                              float64
Playing time_matches played       object
Playing time_starts                int64
Playing time_minutes             float64
Performance_Assists              float64
Performance_non-Penalty Goals    float64
Performance_Penalty Goals        float64
Performance_Yellow Cards         float64
Performance_Red Cards            float64
Expected_xG                      float64
Expected_npxG                    float64
Expected_xAG                     float64
Progression_PrgC                 float64
Progression_PrgP                 float64
Progression_PrgR                 float64
Per 90 minutes_Gls               float64
Per 90 minutes_Ast               float64
Per 90 minutes_G+A               float64
Per 90 minutes_G-PK              float64
Per 90 minutes_G+A-PK            float64
Per 90 minutes_xG                float64
Per 90 minutes_x

In [6]:
df.head()

,Player,Nation,Position,Age,Playing time_matches played,Playing time_starts,Playing time_minutes,Performance_Assists,Performance_non-Penalty Goals,Performance_Penalty Goals,Performance_Yellow Cards,Performance_Red Cards,Expected_xG,Expected_npxG,Expected_xAG,Progression_PrgC,Progression_PrgP,Progression_PrgR,Per 90 minutes_Gls,Per 90 minutes_Ast,Per 90 minutes_G+A,Per 90 minutes_G-PK,Per 90 minutes_G+A-PK,Per 90 minutes_xG,Per 90 minutes_xAG,Per 90 minutes_xG + xAG,Per 90 minutes_npxG,Per 90 minutes_npxG + xAG,Team,Performance_GA,Performance_GA90,Performance_SoTA,Performance_Saves,Performance_Save%,Performance_W,Performance_D,Performance_L,Performance_CS,Performance_CS%,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Penalty Kicks_Save%,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_Sh/90,Standard_SoT/90,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG.1,Expected_npxG.1,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Expected_Ast,Expected_xAG.1,Expected_xA,Expected_A-xAG,Expected_KP,Expected_1/3,Expected_PPA,Expected_CrsPA,Expected_PrgP,Pass Types_Live,Pass Types_Dead,Pass Types_FK,Pass Types_TB,Pass Types_Sw,Pass Types_Crs,Pass Types_TI,Pass Types_CK,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Outcomes_Cmp,Outcomes_Off,Outcomes_Blocks,SCA_SCA,SCA_SCA90,SCA Types_PassLive,SCA Types_PassDead,SCA Types_TO,SCA Types_Sh,SCA Types_Fld,SCA Types_Def,GCA_GCA,GCA_GCA90,GCA Types_PassLive,GCA Types_PassDead,GCA Types_TO,GCA Types_Sh,GCA Types_Fld,GCA Types_Def,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Blocks_Int,Blocks_Tkl+Int,Blocks_Clr,Blocks_Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Starts_Starts,Starts_Mn/Start,Starts_Compl,Subs_Subs,Subs_Mn/Sub,Subs_unSub,Team Success_PPM,Team Success_onG,Team Success_onGA,Team Success (xG)_onxG,Team Success (xG)_onxGA,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%
0,Aaron Cresswell,ENG,"DF,FW",33.0,11.0,4,436.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.4,4.0,26.0,6.0,0.0,0.0,0.0,0.0,0.0,0.00,0.09,0.09,0.00,0.09,West Ham,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,0.0,0.0,0.0,N/a,0.00,0.00,N/a,N/a,N/a,0.0,0.0,0.0,0.0,0.0,N/a,0.0,0.0,254.0,308.0,82.5,4176.0,1722.0,142.0,155.0,91.6,80.0,96.0,83.3,25.0,48.0,52.1,0.0,0.4,0.3,-0.4,5.0,15.0,4.0,0.0,26.0,245.0,63.0,9.0,0.0,4.0,11.0,50.0,4.0,1.0,3.0,0.0,254.0,0.0,3.0,12.0,2.48,7.0,3.0,0.0,0.0,2.0,0.0,1.0,0.21,1.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1.0,1.0,1.0,2.0,3.0,66.7,1.0,5.0,3.0,2.0,3.0,6.0,7.0,0.0,326.0,18.0,125.0,141.0,61.0,1.0,326.0,1.0,0.0,0.0,1.0,100.0,169.0,522.0,220.0,4.0,5.0,0.0,0.0,2.0,210.0,6.0,4,77.0,2,7,18.0,21,1.00,6.0,11.0,6.6,5.6,2.0,3.0,1.0,11.0,0.0,18.0,6.0,3.0,66.7
1,Aaron Hickey,SCO,DF,21.0,9.0,9,713.0,0.0,0.0,0.0,5.0,0.0,0.2,0.2,0.1,9.0,21.0,13.0,0.0,0.0,0.0,0.0,0.0,0.03,0.01,0.04,0.03,0.04,Brentford,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,N/a,0.0,7.0,1.0,14.3,0.88,0.13,0.0,0.0,25.3,0.0,0.0,0.0,0.2,0.2,0.03,-0.2,-0.2,243.0,277.0,87.7,3813.0,894.0,134.0,140.0,95.7,92.0,106.0,86.8,14.0,20.0,70.0,0.0,0.1,0.1,-0.1,1.0,10.0,2.0,0.0,21.0,254.0,23.0,3.0,1.0,3.0,5.0,20.0,0.0,0.0,0.0,0.0,243.0,0.0,7.0,7.0,0.88,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,16.0,9.0,9.0,3.0,4.0,7.0,7.0,100.0,0.0,7.0,3.0,4.0,3.0,19.0,10.0,0.0,356.0,25.0,132.0,155.0,76.0,7.0,356.0,14.0,5.0,35.7,

In [ ]:

df = df.replace('N/a', pd.NA)
columns_to_check = df.columns.difference(['Player', 'Nation', 'Position', 'Team'])
result_df = pd.DataFrame(columns=columns_to_check)
for column in columns_to_check:
    top_players = df[['Player', column]].dropna(subset=[column]).sort_values(by=column, ascending=False).head(3)
    result_df.loc['Top 1', column] = top_players.iloc[0]['Player']
    result_df.loc['Top 2', column] = top_players.iloc[1]['Player'] if len(top_players) > 1 else None
    result_df.loc['Top 3', column] = top_players.iloc[2]['Player'] if len(top_players) > 2 else None
    result_df.loc['Top 1 Value', column] = top_players.iloc[0][column]
    result_df.loc['Top 2 Value', column] = top_players.iloc[1][column] if len(top_players) > 1 else None
    result_df.loc['Top 3 Value', column] = top_players.iloc[2][column] if len(top_players) > 2 else None
result_df


,Aerial Duels_Lost,Aerial Duels_Won,Aerial Duels_Won%,Age,Blocks_Blocks,Blocks_Clr,Blocks_Err,Blocks_Int,Blocks_Pass,Blocks_Sh,Blocks_Tkl+Int,Carries_1/3,Carries_CPA,Carries_Carries,Carries_Dis,Carries_Mis,Carries_PrgC,Carries_PrgDist,Carries_TotDist,Challenges_Att,Challenges_Lost,Challenges_Tkl,Challenges_Tkl%,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Expected_1/3,Expected_A-xAG,Expected_Ast,Expected_CrsPA,Expected_G-xG,Expected_KP,Expected_PPA,Expected_PrgP,Expected_np:G-xG,Expected_npxG,Expected_npxG.1,Expected_npxG/Sh,Expected_xA,Expected_xAG,Expected_xAG.1,Expected_xG,Expected_xG.1,GCA Types_Def,GCA Types_Fld,GCA Types_PassDead,GCA Types_PassLive,GCA Types_Sh,GCA Types_TO,GCA_GCA,GCA_GCA90,Long_Att,Long_Cmp,Long_Cmp%,Medium_Att,Medium_Cmp,Medium_Cmp%,Outcomes_Blocks,Outcomes_Cmp,Outcomes_Off,Pass Types_CK,Pass Types_Crs,Pass Types_Dead,Pass Types_FK,Pass Types_Live,Pass Types_Sw,Pass Types_TB,Pass Types_TI,Penalty Kicks_PKA,Penalty Kicks_PKatt,Penalty Kicks_PKm,Penalty Kicks_PKsv,Penalty Kicks_Save%,Per 90 minutes_Ast,Per 90 minutes_G+A,Per 90 minutes_G+A-PK,Per 90 minutes_G-PK,Per 90 minutes_Gls,Per 90 minutes_npxG,Per 90 minutes_npxG + xAG,Per 90 minutes_xAG,Per 90 minutes_xG,Per 90 minutes_xG + xAG,Performance_Assists,Performance_CS,Performance_CS%,Performance_Crs,Performance_D,Performance_Fld,Performance_Fls,Performance_GA,Performance_GA90,Performance_L,Performance_OG,Performance_Off,Performance_Penalty Goals,Performance_Recov,Performance_Red Cards,Performance_Save%,Performance_Saves,Performance_SoTA,Performance_W,Performance_Yellow Cards,Performance_non-Penalty Goals,Playing time_matches played,Playing time_minutes,Playing time_starts,Progression_PrgC,Progression_PrgP,Progression_PrgR,Receiving_PrgR,Receiving_Rec,SCA Types_Def,SCA Types_Fld,SCA Types_PassDead,SCA Types_PassLive,SCA Types_Sh,SCA Types_TO,SCA_SCA,SCA_SCA90,Short_Att,Short_Cmp,Short_Cmp%,Standard_Dist,Standard_FK,Standard_G/Sh,Standard_G/SoT,Standard_Gls,Standard_PK,Standard_PKatt,Standard_Sh,Standard_Sh/90,Standard_SoT,Standard_SoT%,Standard_SoT/90,Starts_Compl,Starts_Mn/Start,Starts_Starts,Subs_Mn/Sub,Subs_Subs,Subs_unSub,Tackles_Att 3rd,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Tkl,Tackles_TklW,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Team Success (xG)_onxG,Team Success (xG)_onxGA,Team Success_PPM,Team Success_onG,Team Success_onGA,Total_Att,Total_Cmp,Total_Cmp%,Total_PrgDist,Total_TotDist,Touches_Att 3rd,Touches_Att Pen,Touches_Def 3rd,Touches_Def Pen,Touches_Live,Touches_Mid 3rd,Touches_Touches
Top 1,Carlton Morris,James Tarkowski,Łukasz Fabiański,Łukasz Fabiański,James Tarkowski,Joachim Andersen,Wes Foderingham,Antonee Robinson,Alexis Mac Allister,James Tarkowski,João Palhinha,Rodri,Jeremy Doku,Rodri,Mohammed Kudus,Dominic Solanke,Jeremy Doku,Rúben Dias,Rodri,João Palhinha,Mathias Jensen,João Palhinha,Arijanet Muric,Bukayo Saka,Alfie Doughty,Pablo Sarabia,Rodri,Ollie Watkins,Ollie Watkins,Pascal Groß,Phil Foden,Bruno Fernandes,Martin Ødegaard,Rodri,Phil Foden,Erling Haaland,Erling Haaland,Tim Ream,Martin Ødegaard,Son Heung-min,Son Heung-min,Erling Haaland,Erling Haaland,Antony,Anthony Gordon,James Ward-Prowse,Cole Palmer,Ollie Watkins,Anthony Gordon,Anthony Gordon,Oscar Bobb,Jordan Pickford,Jordan Pickford,Brennan Johnson,Lewis Dunk,Virgil van Dijk,Łukasz Fabiański,Tyrick Mitchell,Rodri,Bruno Fernandes,Alfie Doughty,Alfie Doughty,Jordan Pickford,Jordan Pickford,Rodri,Fabian Schär,Martin Ødegaard,Tyrick Mitchell,Jordan Pickford,Jordan Pickford,Đorđe Petrović,Alphonse Areola,Robin Olsen,Manor Solomon,Adama Traoré,Adama Traoré,Sasa Kalajdzic,Sasa Kalajdzic,Kieffer Moore,Kieffer Moore,Kevin De Bruyne,Kieffer Moore,Kieffer Moore,Ollie Watkins,David Raya,Stefan Ortega,Alfie Doughty,Jordan Pickford,Bruno Guimarães,Conor Gallagher,Thomas Kaminski,Ionuț Radu,Thomas Kaminski,Antonee Robinson,Darwin Núñez,Cole Palmer,Bruno Guimarães,Oliver McBurnie,Arijanet Muric,André Onana,Thomas Kaminski,Ederson,João Palhinha,Erling Haa

In [ ]:
df2 = df
df2 = df2.drop(['Nation', 'Position'], axis=1)
team_col = 'Team'
player_col = 'Player'
attribute_columns = [col for col in df2.columns if col not in [team_col, player_col]]
results = {'': ['all']}
for col in attribute_columns:
    results[f'Median of {col}'] = [df2[col].median(skipna=True)]
    results[f'Mean of {col}'] = [df2[col].mean(skipna=True)]
    results[f'Std of {col}'] = [df2[col].std(skipna=True)]
for team in df2[team_col].unique():
    team_df2 = df2[df2[team_col] == team]
    results[''].append(team)
    for col in attribute_columns:
        results[f'Median of {col}'].append(team_df2[col].median(skipna=True))
        results[f'Mean of {col}'].append(team_df2[col].mean(skipna=True))
        results[f'Std of {col}'].append(team_df2[col].std(skipna=True))
results_df = pd.DataFrame(results)
results_df.to_csv('results2.csv', index=False)



c:\Users\admin\anaconda3\envs\ptit\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\admin\anaconda3\envs\ptit\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\admin\anaconda3\envs\ptit\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\admin\anaconda3\envs\ptit\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\admin\anaconda3\envs\ptit\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\admin\anaconda3\envs\ptit\lib\site-packages\numpy\lib\_nanfunctions_imp

In [10]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Columns: 505 entries,  to Std of Aerial Duels_Won%
dtypes: float64(504), object(1)
memory usage: 83.0+ KB


In [ ]:
df3 = df
df3 = df3.drop(['Nation', 'Position'], axis=1)
os.makedirs("histogram", exist_ok=True)
attribute_columns = [col for col in df3.columns if col not in ['Team', 'Player']]
df3[attribute_columns] = df3[attribute_columns].apply(pd.to_numeric, errors='coerce')
for col in attribute_columns:
    plt.figure(figsize=(4, 2.5))
    plt.hist(df3[col].dropna(), bins=20, edgecolor='black')
    plt.title(f'Phân bố {col} - Toàn giải')
    plt.xlabel(col)
    plt.ylabel('Số lượng cầu thủ')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    filename = f"histogram/{col}_ToanGiai.png"
    os.makedirs(os.path.dirname(filename), exist_ok=True) 
    plt.savefig(filename)
    plt.close()

# Vẽ và lưu histogram cho từng đội
for team in df3['Team'].unique():
    team_df3 = df3[df3['Team'] == team]
    for col in attribute_columns:
        plt.figure(figsize=(4, 2.5))
        plt.hist(team_df3[col].dropna(), bins=20, edgecolor='black')
        plt.title(f'Phân bố {col} - Đội {team}')
        plt.xlabel(col)
        plt.ylabel('Số lượng cầu thủ')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        filename = f"histogram/{col}_Doi_{team}.png"
        os.makedirs(os.path.dirname(filename), exist_ok=True)  # Đảm bảo thư mục con tồn tại
        
        # Lưu hình ảnh
        plt.savefig(filename)
        plt.close()


In [ ]:

url = "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats"
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'results2023-202491_overall'})  
    df_overall = pd.read_html(str(table))[0]
df_overall.head()



C:\Users\admin\AppData\Local\Temp\ipykernel_25584\923955945.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_overall = pd.read_html(str(table))[0]


,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,Manchester City,38,28,7,3,96,34,62,91,2.39,80.5,35.6,44.9,1.18,53012,Erling Haaland - 27,Ederson,→ Champions League via league finish
1,2,Arsenal,38,28,5,5,91,29,62,89,2.34,76.1,27.9,48.2,1.27,60236,Bukayo Saka - 16,David Raya,→ Champions League via league finish
2,3,Liverpool,38,24,10,4,86,41,45,82,2.16,87.8,45.7,42.0,1.11,55979,Mohamed Salah - 18,Alisson,→ Champions League via league finish
3,4,Aston Villa,38,20,8,10,76,61,15,68,1.79,63.3,59.9,3.4,0.09,41858,Ollie Watkins - 19,Emiliano Martínez,→ Champions League via league finish
4,5,Tottenham,38,20,6,12,74,61,13,66,1.74,68.2,63.4,4.8,0.13,61482,Son Heung-min - 17,Guglielmo Vicario,→ Europa League via league finish


In [16]:
df_overall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Rk               20 non-null     int64  
 1   Squad            20 non-null     object 
 2   MP               20 non-null     int64  
 3   W                20 non-null     int64  
 4   D                20 non-null     int64  
 5   L                20 non-null     int64  
 6   GF               20 non-null     int64  
 7   GA               20 non-null     int64  
 8   GD               20 non-null     int64  
 9   Pts              20 non-null     int64  
 10  Pts/MP           20 non-null     float64
 11  xG               20 non-null     float64
 12  xGA              20 non-null     float64
 13  xGD              20 non-null     float64
 14  xGD/90           20 non-null     float64
 15  Attendance       20 non-null     int64  
 16  Top Team Scorer  20 non-null     object 
 17  Goalkeeper       2

In [19]:
df_overall['Form'] = df_overall['Pts'] / df_overall['MP']
df_overall.head()

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes,Form
0,1,Manchester City,38,28,7,3,96,34,62,91,2.39,80.5,35.6,44.9,1.18,53012,Erling Haaland - 27,Ederson,→ Champions League via league finish,2.394737
1,2,Arsenal,38,28,5,5,91,29,62,89,2.34,76.1,27.9,48.2,1.27,60236,Bukayo Saka - 16,David Raya,→ Champions League via league finish,2.342105
2,3,Liverpool,38,24,10,4,86,41,45,82,2.16,87.8,45.7,42.0,1.11,55979,Mohamed Salah - 18,Alisson,→ Champions League via league finish,2.157895
3,4,Aston Villa,38,20,8,10,76,61,15,68,1.79,63.3,59.9,3.4,0.09,41858,Ollie Watkins - 19,Emiliano Martínez,→ Champions League via league finish,1.789474
4,5,Tottenham,38,20,6,12,74,61,13,66,1.74,68.2,63.4,4.8,0.13,61482,Son Heung-min - 17,Guglielmo Vicario,→ Europa League via league finish,1.736842


In [24]:
highest_form = df_overall[df_overall["Form"] == df_overall["Form"].max()]
highest_form[['Squad', 'Form']]

,Squad,Form
0,Manchester City,2.394737


### Vậy là đội có phong độ tốt nhất là Manchester City